September 22 2020\
Dannie Fu\
This file converts binary file into csv file and saves the csv

In [164]:
import pandas as pd 
import struct,os,sys
import glob
import csv 

def getTime(startTime, offset, fs):
    return startTime + offset*1000/fs

# Sampling freqs
fs_ACC = 75
fs_BVP = 300
fs_EDA = 15
fs_HRV = 4
fs_STR = 15
fs_TEMP = 15

# Get list of files in session directory 
sessionDir = '/Volumes/Seagate/danslaNature/session-danslanature-2020-09-19_10h25-bin/'
os.chdir(sessionDir)

# Get start time 
sessionFile = sessionDir + 'sessions.csv'
startTime = pd.read_csv(sessionFile, sep=";",usecols=["startTime"])

for file in glob.glob("*[!.csv]"):

    prefix = file.split("._")[0]
    suffix = file.split("._")[1]
    f = open(file,"rb")
    myStruct=struct.Struct(suffix)

    myList = []
    time = []

    i=0
    while True:
        data=f.read(myStruct.size)
        if len(data)==0: break
        a=[*myStruct.unpack(data)]
        #print(str(i)+":",str(a))
        myList.append(a)

        if (file.find('EDA') != -1):
            time.append(getTime(startTime.startTime[0], i, fs_EDA))
        elif (file.find('ACC') != -1):
            time.append(getTime(startTime.startTime[0], i, fs_ACC))
        elif (file.find('BVP') != -1):
            time.append(getTime(startTime.startTime[0], i, fs_BVP))
        elif (file.find('HRV') != -1):
            time.append(getTime(startTime.startTime[0], i, fs_HRV))
        elif (file.find('STR') != -1):
            time.append(getTime(startTime.startTime[0], i, fs_STR))
        elif (file.find('TEMP') != -1 and file.find('TEMPR') == -1):
            time.append(getTime(startTime.startTime[0], i, fs_TEMP))
        i+=1

    if not time:
        df1 = pd.DataFrame(time)
        df2 = pd.DataFrame(myList)
        df = pd.concat([df1, df2],axis=1)
    else:
        df = pd.DataFrame(myList)

    df.to_csv(prefix +".csv", index=False,header=False)  